In [ ]:
%pip install ipympl
import scipy

In [ ]:
from connect4 import connect
from othello import othello
from mtcs import UTCSearch, MCPlay

import numpy as np

%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['toolbar'] = 'None'

from matplotlib import gridspec
import time

def drawprobs(probs):
    ax2.clear()
    a,n,w = zip(*probs)
    an = len(a)
    ax2.bar(np.arange(an),np.array(w)/n)
    ax2.plot([-1,an],[0.5,0.5], color='black',lw=0.5)
    ax2.plot([-1,an],[0,0], color='black',lw=0.5)
    ax2.plot([-1,an],[1,1], color='black',lw=0.5)
    ax2.set_ylim(-0.1,1.1)
    #ax2.set_title(f'{str(sum(n))}')
    ax2.set_axis_off()
    ax3.clear()
    ax3.bar(np.arange(len(a)),n,color='red')
    ax3.set_axis_off()
    refresh()

def drawvals(g):
    ax4.clear()
    ax4.set_ylim(-0.1,1.1)
    ax4.set_axis_off()
    l1 = [v for t,v in g.eval[:g.current] if t == 1]
    l2 = [v for t,v in g.eval[:g.current] if t == -1]
    l = max(len(l1),len(l2),2)
    rx = [0,l-1]
    ax4.plot(rx,[0,0], color='black',lw=0.5,ls='dotted')
    ax4.plot(rx,[0.5,0.5], color='black',lw=0.5,ls='dotted')
    ax4.plot(rx,[1,1], color='black',lw=0.5,ls='dotted')
    ax4.plot(l1, '.-', markersize=10, color='gray')
    ax4.plot(l2, '.-', markersize=10, color='black')

def refresh():
    fig.canvas.draw()
    #fig.canvas.draw_idle()
    #fig.canvas.start_event_loop(0.001)


fig = plt.figure(figsize=(6,6))
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.capture_scroll = False

#plt.subplots_adjust(top=1)
#plt.suptitle('Interactive plot')
#fig.canvas.set_window_title('MCTS')
#fig.canvas.mpl_connect('key_press_event', on_press)
#fig.canvas.mpl_connect('button_press_event', onclick)

def info(msg):
    plt.suptitle(msg)
    fig.canvas.draw()

gs = gridspec.GridSpec(16, 4)
ax  = plt.subplot(gs[4:16,0:4])
ax2 = plt.subplot(gs[1:4,0:1])
ax3 = plt.subplot(gs[1:4,1:2])
ax4 = plt.subplot(gs[1:4,2:4])
ax4.set_axis_off()
plt.tight_layout(pad=0,h_pad=0,w_pad=0)

color = {1:'white', -1: 'black', 0: 'nobody'}

def play():
    global g,current_player, next_player
    while True:
        g = current_player(g)
        current_player, next_player = next_player, current_player
        g.draw(ax)
        drawvals(g)
        refresh()

        if g.terminal:
            info(f'{color[g.winner]} ({engine[g.winner]}) wins!')
            break
        if g.resigned:
            info(f'{color[g.turn]} ({engine[g.turn]}) resigned')
            break



games = {'othello': (othello, 8),
         'connect': (connect, 7, 6, 4)}

players = {'mcts':  lambda g: UTCSearch(g, 10000, 5, drawprobs),
           'pure':  lambda g: MCPlay(g, 100),
           'human': lambda g: Human(g)}

white = players['mcts']
black = players['pure']
engine = {1: 'mcts', -1: 'pure', 0: 'nobody'}

current_player = white
next_player    = black

g = games['connect']
g = g[0](*g[1:])

g.draw(ax)
info('MTCS')

In [ ]:
play()